In [22]:
obj='SN2019yvr'
path='../DATA/'+obj+'/'
FILENAME = path+'20200108T094303_rp.fits'
date_use=FILENAME.split('/')[-1][:8]
fname_alt=FILENAME.split('/')[-1].split('.')[-2]
ra,dec=191.191856,-0.513369

In [24]:
MAGNITUDE_LIMIT = -1.5 # arbitrary
FWHM = 6 # pixels, First Guess, will be derived in script.
FONT_SIZE = 14
COORDINATES = path+'cat/localstandards_20200125.dat'
NEW_FILENAME= path+'cat/localstandards_'+date_use+'.dat'

PHOT_FILENAME = path+'phot/phot_'+fname_alt+'.dat'

import os
os.makedirs(path+'cat/', exist_ok=True)
os.makedirs(path+'phot/', exist_ok=True)

verbose=False
saveseq=False

### Make local standards file if saveseq=True, else just derive the FWHM.

This method is very cumbersome for just deriving the FWHM. It tries to reject galaxies, cosmics, and trails/streaks before deriving fwhm. overkill. Also annoyingly prints output. Should change to something simpler and more efficient later.

In [4]:
import numpy as np

from astropy.wcs import WCS
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.stats import sigma_clip
from astropy.table import Table
from photutils import DAOStarFinder

import matplotlib.pyplot as plt

from astropy.nddata import NDData
from astropy.modeling.fitting import LevMarLSQFitter

from photutils import CircularAperture, CircularAnnulus
from photutils.psf import extract_stars
from photutils import aperture_photometry, EPSFBuilder, EPSFFitter
from photutils.psf import BasicPSFPhotometry, DAOGroup
from photutils.background import MMMBackground

from imexam.imexamine import Imexamine
from astropy.stats import gaussian_sigma_to_fwhm
from astropy.visualization import simple_norm

import pandas as pd

from astropy.coordinates import SkyCoord
import calviacat as cvc



    

coord_SN=SkyCoord(ra,dec,unit=('deg', 'deg'), frame='icrs')

def rm_galaxies(ic,sources,verbose=False,retdf=False):
    
    df=pd.DataFrame(index=ic,columns=['s','r1','r2'])
    sp=[]
    rp1=[]
    rp2=[]
    for i in ic:
        sp.append(sources[i]['sharpness'])
        rp1.append(sources[i]['roundness1'])
        rp2.append(sources[i]['roundness2'])
    df['s']=sp
    df['r1']=rp1
    df['r2']=rp2

    df['delta_s']=df.s-df.s.mean()
    df['delta_r1']=df.r1-df.r1.mean()
    df['delta_r2']=df.r2-df.r2.mean()

    df['del_r2']=(df.delta_r1**2+df.delta_r2**2)**(1/2)

    maskeds=sigma_clip(df['s'],sigma=2.5)
    #maskedr1=sigma_clip(df['s'],sigma=2.5)
    #maskedr2=sigma_clip(df['s'],sigma=2.5)

    idx_rm=df[((df.delta_s<-0.1) & (df.del_r2>df.del_r2.median())) | (maskeds.mask==True)].index.values
    
    
    
    
    if verbose:
        print(idx_rm)
    
    if not retdf:
        if len(idx_rm)==0:
            return ic

        for idx in idx_rm:
            if verbose:
                print(idx)
            ic.remove(idx)
        return ic

    if retdf:
        if len(idx_rm)==0:
            return ic,df

        for idx in idx_rm:
            if verbose:
                print(idx)
            ic.remove(idx)
        return ic,df
        

###Get image and WCS, find stars, remove galaxies
image = type('image', (object,), dict()) # image container

# get image and wcs solution
with fits.open(FILENAME) as hdul:
    image.image = hdul[0].data
    image.wcs = WCS(hdul[0].header)
    image.mask= hdul[2].data.astype(bool)
    #print(hdul[2].data.astype(bool))
    bintab= hdul[1].data
    binhead= hdul[1].header
    image.clean=np.ma.masked_array(image.image,image.mask)

    
    
# get find sources in the image
image.background, std = sigma_clipped_stats(image.image, sigma=3,maxiters=10,mask=image.mask)[1:]
image.subclean=image.clean-image.background
sources = DAOStarFinder(fwhm=FWHM, threshold=7.*std,exclude_border=True,
                        sharphi=0.8,sigma_radius=1.1,peakmax=60000)(image.subclean)
ic = [i for i, row in enumerate(sources) if np.isfinite(row['mag']) and row['mag'] <= MAGNITUDE_LIMIT]
ic,df=rm_galaxies(ic,sources,retdf=True) 
image.x, image.y, image.mag = sources['xcentroid'][ic], sources['ycentroid'][ic], sources['mag'][ic]
image.ra, image.dec = image.wcs.wcs_pix2world(image.x, image.y, 0)


##Get all local stars, remove FWHM too high.
image.stars = list()


plots=Imexamine()
fwhms=np.ones(len(image.x))
for i, (x, y) in enumerate(zip(image.x, image.y)):
    try:
        _fwhm_=(plots.line_fit(x,y,image.subclean,genplot=False).stddev_0 * gaussian_sigma_to_fwhm)
        fwhms[i]=_fwhm_
    except:
        fwhms[i]=-100.0

mask=(fwhms<3.5) | (fwhms>11.0) 
masked_fwhms=np.ma.MaskedArray(fwhms,mask)
        
fwhms_clean=sigma_clip(masked_fwhms,maxiters=20,sigma=2.0)
fwhm=np.int(np.round(np.mean(fwhms_clean),0))



X, Y, Z = *np.meshgrid(*map(np.arange, image.image.shape[::-1])), image.image - image.background
for x, y in zip(np.round(image.x).astype(int), np.round(image.y).astype(int)):
    i = slice(y - FWHM, y + FWHM + 1), slice(x - FWHM, x + FWHM + 1)
    #x, y = np.round( (Z[i] * (X[i], Y[i])).sum(axis=(1,2)) / Z[i].sum() ).astype(int)
    #i = slice(y - FWHM, y + FWHM + 1), slice(x - FWHM, x + FWHM + 1)
    image.stars.append(i)
    
local_sequence=np.where(~fwhms_clean.mask)[0]

if saveseq:
    np.savetxt(NEW_FILENAME, list(zip(image.ra[local_sequence], image.dec[local_sequence])))




using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=372.020520	yc=38.546058
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=375.538271	yc=71.885059
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')


/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


xc=425.583054	yc=92.093955
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=1969.814659	yc=231.458551
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=2267.603080	yc=627.818883
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=3987.535082	yc=739.879262
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=94.649510	yc=948.680536
using model: <class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev')
xc=807.18

## Construct ePSF, calculate App. Phot and PSF Phot.

Takes a bit of time to construct ePSF and extract PSF Phot. This is also where we would want to add template subtraction in the future.

In [16]:

 # ra, dec

FONT_SIZE = 16

FWHM = fwhm # pixels
ANNULUS_RADII = FWHM+2, FWHM+5 # pixels
OVERSAMPLING = 1 # factor




image = type('image', (object,), dict())
with fits.open(FILENAME) as hdul:
    image.image = hdul[0].data
    image.header = hdul[0].header
    image.mask= hdul[2].data.astype(bool)
    image.clean=np.ma.masked_array(image.image,image.mask)
image.wcs = WCS(image.header)

image.background, std = sigma_clipped_stats(image.image, sigma=3,maxiters=10,mask=image.mask)[1:]
image.subclean=image.clean-image.background

filter_dictionary={'rp':'r','gp':'g','ip':'i','zp':'z','up':'u',
                   'B':'B','V':'V','R':'R'}



coordinates = type('coordinates', (object,), dict())
coordinates.ra, coordinates.dec = np.loadtxt(COORDINATES, unpack=True, usecols=(0,1))
coordinates.x, coordinates.y = zip(*image.wcs.wcs_world2pix(list(zip(*(coordinates.ra, coordinates.dec))), 0))


image.stars = [
    (slice(y - FWHM, y + FWHM + 1), slice(x - FWHM, x + FWHM + 1))
    for x, y in zip(np.round(coordinates.x).astype(int), np.round(coordinates.y).astype(int))
]

image.centered_stars = image.stars.copy()
coordinates.centered_x = list(coordinates.x).copy()
coordinates.centered_y = list(coordinates.y).copy()


##CENTER STARS
n=0
max_repeat_center=10
while True:
    X, Y, Z = *np.meshgrid(*map(np.arange, image.image.shape[::-1])), image.subclean
    delta=0
    for i, star in enumerate(image.centered_stars):

        x, y = (Z[star] * (X[star], Y[star])).sum(axis=(1,2)) / Z[star].sum()
        delta += np.sqrt((coordinates.centered_x[i] - x)**2 + (coordinates.centered_y[i] - y)**2)
        coordinates.centered_x[i], coordinates.centered_y[i] = x, y

        x, y = np.round((x, y)).astype(int)
        star = slice(y - FWHM, y + FWHM + 1), slice(x - FWHM, x + FWHM + 1)
        image.centered_stars[i] = star
    if verbose: 
        print('Delta is:',delta)
    if delta<=0.1:
        break
    n+=1
    if n>max_repeat_center:
        break
if verbose:       
    print('Successfully centered Stars in apertures')

apertures = CircularAperture(zip(coordinates.centered_x, coordinates.centered_y), FWHM)
annuli = CircularAnnulus(zip(coordinates.centered_x, coordinates.centered_y), *ANNULUS_RADII)

#fig = plt.figure(figsize=(20, 2 * (len(image.stars) // 10 + 1)))
#for i in range(len(image.stars)):
    
    #aperture, annulus = apertures.to_mask()[i], annuli.to_mask()[i]
    
    #ax = fig.add_subplot(len(image.stars) // 10 + 1, 10, i + 1)
    #ax.text(0, 0, i, color='black', fontsize=14, transform=ax.transAxes)

    #img = aperture.cutout(image.image)
    #img = plt.get_cmap('viridis')((img - img.min()) / (img.max() - img.min()))
    #img[:,:,3] = aperture.data
    #plt.imshow(img, origin='lower', extent=aperture.bbox.extent)

    #img = annulus.cutout(image.image)
    #img = plt.get_cmap('viridis')((img - img.min()) / (img.max() - img.min()))
    #img[:,:,3] = annulus.data
    #plt.imshow(img, origin='lower', extent=annulus.bbox.extent)

    #apertures.plot(indices=i, color='white')
    
#fig.tight_layout()

#Make stars to fit EPSF
fix_position = ()
curr_fitter=EPSFFitter(fit_boxsize=4)

stars_for_epsf=range(len(image.stars))
#stars_for_epsf

xd=(image.image-image.background)
mask2=xd<0.0

stars = extract_stars(
    NDData(data=image.image - image.background,mask=image.mask|mask2),
    Table([
        dict(x=coordinates.centered_x[i], y=coordinates.centered_y[i])
        for i in stars_for_epsf
    ]),
    size = 29
)

#Fit PSF
image.epsf = EPSFBuilder(oversampling=OVERSAMPLING,
                                 maxiters=500,fitter=curr_fitter)(stars)[0]

if verbose:       
    print('Successfully built PSF model')

profile = image.epsf.data.sum(axis=0)
itop = profile.argmax()

left = np.argmin((profile[:itop] - profile.max()/2)**2)
right = np.argmin((profile[itop:] - profile.max()/2)**2) + itop

FWHM=np.ceil((right - left) / OVERSAMPLING).astype(int)

FWHM=int(FWHM)

#APERTURE PHOT
coordinates.aperture = list()

error = image.background + image.image - image.background
photometry = aperture_photometry(image.image, [apertures, annuli], error=np.sqrt(image.image))

for phot in photometry:
    
    flux, error = phot['aperture_sum_0'], phot['aperture_sum_err_0']
    
    flux = flux - phot['aperture_sum_1'] / annuli.area() * apertures.area()
    error = np.sqrt(error**2 + (phot['aperture_sum_err_1'] * apertures.area() / annuli.area())**2)

    coordinates.aperture.append((flux, error))

if verbose:       
    print('App. Phot Success')
#PSF PHOT

coordinates.psf = list()

photometry = BasicPSFPhotometry(
    group_maker = DAOGroup(FWHM),
    bkg_estimator = MMMBackground(),
    psf_model = image.epsf,
    fitter = LevMarLSQFitter(),
    fitshape = 29,
    aperture_radius = FWHM
)

for i, star in enumerate(image.centered_stars):
    
    image.epsf.fixed.update(dict(x_0=False, y_0=False))

    res = photometry(
        image = image.image - image.background,
        init_guesses=Table([
            dict(x_0=coordinates.centered_x[i], y_0=coordinates.centered_y[i])
        ])
    )
    coordinates.psf.append((res['flux_fit'].data[0], res['flux_unc'].data[0]))

if verbose:       
    print('Psf Phot Success')

data = np.vstack([coordinates.ra, coordinates.dec, *zip(*coordinates.psf), *zip(*coordinates.aperture)])
#header = 'aperture = %d, annulus = %d, %d' % (FWHM, *ANNULUS_RADII)
header ='ra dec psf psf_err ap ap_err'
np.savetxt(PHOT_FILENAME, np.transpose(data), header=header,comments='#')


Delta is: 38.13357547715194
Delta is: 9.318061300967015
Delta is: 2.0304088105385567
Delta is: 0.0
Successfully centered Stars in apertures
Successfully built PSF modelmax 500) [0.2 s/iter]
App. Phot Success


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:134: RuntimeWarning: invalid value encountered in sqrt


Psf Phot Success


## Calibrate 

Calibrates using "calviacat" shortcut script using mastcasjobs to fetch PanSTARRS1 catalog data.

Currently: uncertain what to do if filter is uBV. Also using PS1 instead of Refcat. Refcat is returning empty catalog in calviacat sometimes, cannot figure out why.

In [25]:
df=pd.read_csv(PHOT_FILENAME,delim_whitespace=True,escapechar='#')
AT=Table.from_pandas(df)
c=SkyCoord(AT['ra'],AT['dec'],unit=('deg', 'deg'), frame='icrs')
i_rm=coord_SN.match_to_catalog_sky(c)[0] #Location of SN

wsid='1844987713'
password='jeknun-dosxaw-7cAvdo'

ps1 = cvc.PanSTARRS1(path+'cat/cat'+fname_alt+'.db')
ps1.fetch_field(c)

objids, distances = ps1.xmatch(c)


i_inst = -2.5 * np.log10(AT['psf']/image.header['EXPTIME'])
i_err = AT['psf_err'] / AT['psf']/image.header['EXPTIME'] * 1.0857

filtername=filter_dictionary[image.header['FILTER']]

zp_mean,zp_med,zp_err,m,gmi_constant=ps1.cal_constant(objids, i_inst,filtername,mlim=[13,18])

F_inst=(i_inst+zp_med)
F_inst_err=np.sqrt(i_err**2 + zp_err**2)


Fetching PS1 catalog from STScI over 0.29 deg field-of-view.
Updating panstarrs1 with 3000 sources.
Matched 38 sources to photometric catalog.


## Save photometry

"simple" way to add new photometry of the object into photometry pandas dataframe saved as a csv

In [26]:
mjd=image.header['MJD-OBS']
row_dict={'mjd':mjd,'mag':F_inst[i_rm],'err':F_inst_err[i_rm],'filter':filtername}

final_phot_file=path+'phot/photdf.csv'
if os.path.exists(final_phot_file):
    photdf=pd.read_csv(final_phot_file)
    
    if (~np.any(photdf.mjd==mjd)):
    #if True:
        photdf_newrow=pd.DataFrame({'mjd':mjd,'mag':F_inst[i_rm],'err':F_inst_err[i_rm],'filter':filtername},index=[0])
        photdf=photdf.append(photdf_newrow)
    else:
        row_change=np.where(photdf.mjd==mjd)[0][0]
        photdf_newrow=pd.DataFrame({'mjd':mjd,'mag':F_inst[i_rm],'err':F_inst_err[i_rm],'filter':filtername},index=[row_change])
        
        photdf.iloc[row_change]=(mjd,F_inst[i_rm],F_inst_err[i_rm],filtername)
        
else:
    photdf=pd.DataFrame({'mjd':mjd,'mag':F_inst[i_rm],'err':F_inst_err[i_rm],'filter':filtername},index=[0])
    
photdf.to_csv(final_phot_file,index=False)
photdf=pd.read_csv(final_phot_file)
photdf.sort_values('mjd')